# Cluster formations

I only had a couple days to work on this over the holiday, but I wanted to upload.

1. Invert R to L plays
2. Line of scrimmage abs val xy coordinates
3. Shrink field
4. Fill players on new field 

Detect Offensive and Defensive formations for each play type (kickoff, punt, extra point, and field goal)

In [ ]:
%%capture
!conda install -y -c conda-forge hdbscan umap-learn;

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import hdbscan
import seaborn as sns
from umap import UMAP


In [ ]:
game_data = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2022/games.csv")
plays = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2022/plays.csv")
scouting = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv")


In [ ]:
punt_plays = plays[plays["specialTeamsPlayType"] == "Punt"].copy()


In [ ]:
# merge game data
punt_plays = punt_plays.merge(game_data, left_on="gameId", right_on="gameId")

In [ ]:
# merge game data
all_plays = plays.merge(game_data, left_on="gameId", right_on="gameId")

In [ ]:
# merge scouting data
all_plays = all_plays.merge(scouting, left_on=["gameId", "playId"], right_on=["gameId", "playId"])

In [ ]:
# Add in tracking for 18, 19, 20 and filter to the first frame only
iter_csv = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv", iterator=True, chunksize=1e6)
tracking18 = pd.concat([chunk[chunk["frameId"] == 1] for chunk in iter_csv])

iter_csv = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv", iterator=True, chunksize=1e6)
tracking19 = pd.concat([chunk[chunk["frameId"] == 1] for chunk in iter_csv])

iter_csv = pd.read_csv("/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv", iterator=True, chunksize=1e6)
tracking20 = pd.concat([chunk[chunk["frameId"] == 1] for chunk in iter_csv])


In [ ]:
formation_tracking = pd.concat([tracking18, tracking19, tracking20])


In [ ]:
del tracking18, tracking19, tracking20

In [ ]:
df = formation_tracking.merge(all_plays, on=["gameId", "playId"])


In [ ]:
del all_plays

In [ ]:
df.loc[df["y"] < 0, "y"] = 0
df.loc[df["y"] > 53, "y"] = 53

In [ ]:
# Normalize the data
df["x_norm"] = df["x"]
df["y_norm"] = df["y"]
df["absoluteYardlineNumber_norm"] = df["absoluteYardlineNumber"]

df.loc[df["playDirection"] == "left", "x_norm"] = (
    120 - df.loc[df["playDirection"] == "left", "x"]
)
df.loc[df["playDirection"] == "left", "y_norm"] = (
    53 - df.loc[df["playDirection"] == "left", "y"]
)
df.loc[df["playDirection"] == "left", "absoluteYardlineNumber_norm"] = (
    120 - df.loc[df["playDirection"] == "left", "absoluteYardlineNumber"]
)

# Assign possession

In [ ]:
def has_possession(row):
    if (row["team"] == "away") and (
        row["possessionTeam"] == row["visitorTeamAbbr"]
    ):
        return True
    elif (row["team"] == "home") and (
        row["possessionTeam"] == row["homeTeamAbbr"]
    ):
        return True
    else:
        return False


In [ ]:
df["has_possession"] = df.apply(has_possession, axis=1)


# Normalize x from line of scrimmage and y from ball

In [ ]:
# should be dist from line of scrimmage
df["x_from_scrimmage"] = df["absoluteYardlineNumber_norm"] - df["x_norm"]

In [ ]:
def calculate_y_from_ball(rows):
    y_football = rows.loc[rows["team"] == "football", "y_norm"].item()
    return rows["y_norm"] - y_football

In [ ]:
df["y_from_ball"] = df.groupby(["gameId", "playId"]).apply(calculate_y_from_ball).droplevel(level=[0,1])

In [ ]:
df.loc[df["has_possession"], "x_from_scrimmage"].hist(bins=20)

In [ ]:
df.loc[~df["has_possession"], "x_from_scrimmage"].hist(bins=20)

In [ ]:
# offensive players should all be greater than 0 at the start of the play
# some aren't, let's set the ones greater than a yard off to 0
df.loc[
    (df["x_from_scrimmage"] < -1) & (df["has_possession"]), "x_from_scrimmage"
] = 0

# defensive players should all be less than 0 at the start of the play
# some aren't, let's set the ones greater than a yard off to 0
df.loc[
    (df["x_from_scrimmage"] > 1) & (~df["has_possession"]), "x_from_scrimmage"
] = 0

df["x_from_scrimmage_abs"] = df["x_from_scrimmage"].abs()


# Separate offense and defense

In [ ]:
# defense
defense_df = df.loc[~df["has_possession"]].copy()

# offense
offense_df = df.loc[df["has_possession"]].copy()


# Normalize the x,y coordinates

In [ ]:
def_min_x = defense_df["x_from_scrimmage_abs"].min()
def_max_x = defense_df["x_from_scrimmage_abs"].max()

off_min_x = offense_df["x_from_scrimmage_abs"].min()
off_max_x = offense_df["x_from_scrimmage_abs"].max()

print(def_min_x, def_max_x)
print(off_min_x, off_max_x)

In [ ]:
x_dim = 100 // 2 
y_dim = 53 // 2

defense_x_range = np.linspace(def_min_x, def_max_x, int(x_dim))
offense_x_range = np.linspace(off_min_x, off_max_x, int(x_dim))

y_range = np.linspace(df["y_from_ball"].min(), df["y_from_ball"].max(), y_dim)


In [ ]:
def get_closest(array, values):
    # make sure array is a numpy array
    array = np.array(array)

    # get insert positions
    idxs = np.searchsorted(array, values, side="left")

    # find indexes where previous index is closer
    prev_idx_is_less = (idxs == len(array)) | (
        np.fabs(values - array[np.maximum(idxs - 1, 0)])
        < np.fabs(values - array[np.minimum(idxs, len(array) - 1)])
    )
    idxs[prev_idx_is_less] -= 1

    # this returns the closest value from linspace
    #return array[idxs]

    # this returns the index of the closest value from linspace, which is really all we need, I think
    return idxs


In [ ]:
defense_df["x_transformed"] = get_closest(
    defense_x_range, defense_df["x_from_scrimmage_abs"]
)
defense_df["y_transformed"] = get_closest(y_range, defense_df["y_norm"])

offense_df["x_transformed"] = get_closest(
    offense_x_range, offense_df["x_from_scrimmage_abs"]
)
offense_df["y_transformed"] = get_closest(y_range, offense_df["y_norm"])


# Populate an empty array (field) with players' norm x,y

In [ ]:
# Set the field to be a grid
empty_field = np.zeros((y_dim, x_dim))

In [ ]:
def populate_field(gbdf, binary=False):
    """returns an array (field) with the dimensions (x_dim, y_dim)
    populated with players locations at each x,y coordinate

    Args:
        gbdf pandas DataFrame: a groupby dataframe of the play dataframe
        binary bool: if True, will return a binary field with 1s where players are, else sum up the number of players
    """
    # remove the football from the field
    gbdf = gbdf.loc[gbdf["team"] != "football"]
    # copy the empty field to populate it
    populated_field = empty_field.copy()
    x_idxs = gbdf["x_transformed"].values
    y_idxs = gbdf["y_transformed"].values
    if binary is True:
        populated_field[y_idxs, x_idxs] = 1
    else:
        # results are accumulated for results w/ more than one index :mind_blown:
        np.add.at(populated_field, (y_idxs, x_idxs), 1)

    # assert populated_field.sum() == 11
    # if populated_field.sum() != 11:
    #     print(populated_field.sum())
    return populated_field.flatten()

In [ ]:
# boolean field
defense_sparse = defense_df.loc[defense_df["team"] != "football"].groupby(["gameId", "playId"]).apply(populate_field, binary=True)
offense_sparse = offense_df.loc[offense_df["team"] != "football"].groupby(["gameId", "playId"]).apply(populate_field, binary=True)

In [ ]:
# How lossy was the transformation to subspace?
# Of ~20,000 plays, how many mapped all 11 players uniquely?

# this was from .sum()
defense_sparse.apply(lambda row: row.sum()).value_counts()

In [ ]:
# How lossy was the transformation to subspace?
# Of ~20,000 plays, how many mapped all 11 players uniquely?
offense_sparse.apply(lambda row: row.sum()).value_counts()

In [ ]:
# field where the counts represent the number of players at each x,y coordinate
defense_sparse = defense_df.loc[defense_df["team"] != "football"].groupby(["gameId", "playId", "specialTeamsPlayType", "has_possession"]).apply(populate_field, binary=False)
offense_sparse = offense_df.loc[offense_df["team"] != "football"].groupby(["gameId", "playId", "specialTeamsPlayType", "has_possession"]).apply(populate_field, binary=False)

In [ ]:
# better be all 11
defense_sparse.apply(lambda row: row.sum()).value_counts()

In [ ]:
# better be all 11 
offense_sparse.apply(lambda row: row.sum()).value_counts()

In [ ]:
defense_play_arrays = defense_sparse.copy()
offense_play_arrays = offense_sparse.copy()

In [ ]:
# name the pandas series so we can merge them
defense_play_arrays.name = "normalized_player_locations"
offense_play_arrays.name = "normalized_player_locations"

In [ ]:
defense_play_arrays = defense_play_arrays.to_frame()
offense_play_arrays = offense_play_arrays.to_frame()

# Reduce the dimensionality of the populated field and cluster the result

In [ ]:
defense_players_df = pd.DataFrame(
    defense_play_arrays["normalized_player_locations"].tolist(),
    index=defense_play_arrays.index,
)
offense_players_df = pd.DataFrame(
    offense_play_arrays["normalized_player_locations"].tolist(),
    index=offense_play_arrays.index,
)


In [ ]:
# PUNT
defense_punt = defense_players_df.loc[
    defense_players_df.index.get_level_values("specialTeamsPlayType") == "Punt"
]
offense_punt = offense_players_df.loc[
    offense_players_df.index.get_level_values("specialTeamsPlayType") == "Punt"
]

# KICKOFF
defense_kickoff = defense_players_df.loc[
    defense_players_df.index.get_level_values("specialTeamsPlayType")
    == "Kickoff"
]
offense_kickoff = offense_players_df.loc[
    offense_players_df.index.get_level_values("specialTeamsPlayType")
    == "Kickoff"
]

# EXTRA POINT
defense_extra_point = defense_players_df.loc[
    defense_players_df.index.get_level_values("specialTeamsPlayType")
    == "Extra Point"
]
offense_extra_point = offense_players_df.loc[
    offense_players_df.index.get_level_values("specialTeamsPlayType")
    == "Extra Point"
]

# FIELD GOAL
defense_field_goal = defense_players_df.loc[
    defense_players_df.index.get_level_values("specialTeamsPlayType")
    == "Field Goal"
]
offense_field_goal = offense_players_df.loc[
    offense_players_df.index.get_level_values("specialTeamsPlayType")
    == "Field Goal"
]


In [ ]:
def umapify(
    play_type_df,
    umap__n_neighbors=133,
    hdbscan__min_cluster_size=50,
    hdbscan__min_samples=15,
):
    """Apply UMAP and HDBSCAN to a DataFrame of player locations

    Args:
        play_type_df (DataFrame): A DataFrame of normalized player locations

    Returns:
        pandas DataFrame: A DataFrame of player locations in UMAP space
    """

    umap = UMAP(
        n_neighbors=umap__n_neighbors,
        min_dist=0,
        metric="euclidean",
        n_components=2,
    )

    umapdf = umap.fit_transform(play_type_df)
    umapdf = pd.DataFrame(umapdf, index=play_type_df.index)
    umapdf.columns = ["component_1", "component_2"]


    umapdf = umapdf.merge(
        # note this is the original df, which has play type
        # keeps only one record per play, which is fine for play-level data
        df.drop_duplicates(subset=["gameId", "playId"]).set_index(
            ["gameId", "playId", "specialTeamsPlayType", "has_possession"]
        )[
            ["specialTeamsResult"]
            + scouting.drop(columns=["gameId", "playId"]).columns.tolist()
        ],
        left_index=True,
        right_index=True,
        how="left",
    )

    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan__min_samples,
        min_cluster_size=hdbscan__min_cluster_size,
    ).fit_predict(umapdf[["component_1", "component_2"]])

    umapdf["cluster"] = labels

    return umapdf


In [ ]:
defense_punt_umap = umapify(
    defense_punt,
    umap__n_neighbors=55,
    hdbscan__min_cluster_size=15,
    hdbscan__min_samples=5,
)
offense_punt_umap = umapify(
    offense_punt,
    umap__n_neighbors=75,
    hdbscan__min_cluster_size=35,
    hdbscan__min_samples=15,
)

defense_kickoff_umap = umapify(
    defense_kickoff,
    umap__n_neighbors=75,
    hdbscan__min_cluster_size=75,
    hdbscan__min_samples=15,
)
offense_kickoff_umap = umapify(
    offense_kickoff,
    umap__n_neighbors=75,
    hdbscan__min_cluster_size=75,
    hdbscan__min_samples=15,
)

defense_extra_point_umap = umapify(
    defense_extra_point,
    umap__n_neighbors=500,
    hdbscan__min_cluster_size=50,
    hdbscan__min_samples=15,
)
offense_extra_point_umap = umapify(
    offense_extra_point,
    umap__n_neighbors=75,
    hdbscan__min_cluster_size=50,
    hdbscan__min_samples=15,
)

defense_field_goal_umap = umapify(
    defense_field_goal,
    umap__n_neighbors=150,
    hdbscan__min_cluster_size=50,
    hdbscan__min_samples=15,
)
offense_field_goal_umap = umapify(
    offense_field_goal,
    umap__n_neighbors=75,
    hdbscan__min_cluster_size=50,
    hdbscan__min_samples=15,
)


In [ ]:
def plot_clusters(umapdf):
    fig, ax = plt.subplots(figsize=(8, 8))
    for cluster in sorted(umapdf.cluster.unique()):
        ax.scatter(
            umapdf.loc[umapdf.cluster == cluster, "component_1"],
            umapdf.loc[umapdf.cluster == cluster, "component_2"],
            s=5,
            alpha=0.5,
            label=cluster,
            color=sns.color_palette("tab20")[cluster],
        )
    # ax.grid();
    ax.legend();

    if umapdf.index.get_level_values("has_possession").unique()[0]:
        side = "Offense"
    else:
        side = "Defense"
    n_plays = umapdf.shape[0]
    play_type = umapdf.index.get_level_values("specialTeamsPlayType").unique()[0]
    ax.set_title(f"Clusters of {n_plays} {side} {play_type} Plays");

In [ ]:
for play_type_umap in [
    defense_punt_umap,
    offense_punt_umap,
    defense_kickoff_umap,
    offense_kickoff_umap,
    defense_extra_point_umap,
    offense_extra_point_umap,
    defense_field_goal_umap,
    offense_field_goal_umap,
]:  
    plot_clusters(play_type_umap)


# Visualize the mean formation in the subspace (normed field)

In [ ]:
play_type_dfs = [
    defense_punt,
    offense_punt,
    defense_kickoff,
    offense_kickoff,
    defense_extra_point,
    offense_extra_point,
    defense_field_goal,
    offense_field_goal,
]
play_type_df_umaps = [
    defense_punt_umap,
    offense_punt_umap,
    defense_kickoff_umap,
    offense_kickoff_umap,
    defense_extra_point_umap,
    offense_extra_point_umap,
    defense_field_goal_umap,
    offense_field_goal_umap,
]

for play_type_df, play_type_df_umap in zip(play_type_dfs, play_type_df_umaps):
    has_possession = play_type_df.index.get_level_values("has_possession").unique()[0]
    play_type = play_type_df.index.get_level_values("specialTeamsPlayType").unique()[0]
    if has_possession:
        side = "Offense"
    else:
        side = "Defense"

    clusters = play_type_df.merge(
    play_type_df_umap["cluster"], left_index=True, right_index=True
    )
    clusters_reshaped = clusters.groupby("cluster").apply(
        lambda row: row[range(1300)].mean()
    )
    clusters_reshaped = clusters_reshaped[
        range(1300)
    ].apply(lambda row: row.values.reshape(y_dim, x_dim), axis=1)

    for cluster, cluster_array in clusters_reshaped.iteritems():
        fig, ax = plt.subplots(figsize=(10, 4))
        ax.set_title(f"{play_type} {side} Cluster {cluster} ({clusters['cluster'].value_counts()[cluster]} Plays)")
        sns.heatmap(cluster_array, ax=ax)

# Visualize a few plays in each cluster

In [ ]:
# from svgpathtools import svg2paths
# from svgpath2mpl import parse_path
# football, football_attributes = svg2paths("american-football.svg")
# football_marker = parse_path(football_attributes[0]["d"])


In [ ]:
def plot_play(gameid, playid, normdf):
    play = normdf.loc[(normdf["gameId"] == gameid) & (normdf["playId"] == playid)]

    fig = plt.figure(figsize=(12, 5))
    ax = plt.axes(xlim=(0, 120), ylim=(0, 53))


    football = play.loc[play["team"] == "football"]

    ax.scatter(play["x_from_scrimmage_abs"], play["y_norm"], color="#fb8500", s=150)
    ax.scatter(
        football["x_from_scrimmage_abs"],
        football["y_norm"],
        marker='^',
        color="brown",
        s=250,
    )

    ax.set_yticks([])
    ax.set_xticks([30, 60, 90])
    ax.set_xticklabels(["20", "50", "20"])
    ax.vlines(10, 0, 53.3, color="black", linestyle="-", alpha=0.5)
    ax.vlines(60, 0, 53.3, color="black", linestyle="-", alpha=0.5)
    ax.vlines(110, 0, 53.3, color="black", linestyle="-", alpha=0.5)


    

In [ ]:
for cluster in sorted(offense_extra_point_umap.cluster.unique()):
    print(f"Cluster {cluster}")
    cluster_plays = offense_extra_point_umap.loc[offense_extra_point_umap["cluster"] == cluster].reset_index()
    gameids = cluster_plays["gameId"]
    playids = cluster_plays["playId"]
    
    for i, (gameid, playid) in enumerate(zip(gameids, playids)):
        if i == 5:
            break
        plot_play(gameid, playid, defense_df)


# Insights from clusters?

## Kickoff -- formation dectection comparison offense

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(
    offense_kickoff_umap.loc[
        offense_kickoff_umap["kickoffReturnFormation"] == "8-0-2", "component_1"
    ],
    offense_kickoff_umap.loc[
        offense_kickoff_umap["kickoffReturnFormation"] == "8-0-2", "component_2"
    ],
    s=5,
    alpha=0.5,
    color="lightgrey",
    label="8-0-2",
)
ax.scatter(
    offense_kickoff_umap.loc[
        offense_kickoff_umap["kickoffReturnFormation"] == "8-0-1", "component_1"
    ],
    offense_kickoff_umap.loc[
        offense_kickoff_umap["kickoffReturnFormation"] == "8-0-1", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="8-0-1",
)

# Hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.legend();


## Kickoff - kickType defense

In [ ]:
defense_kickoff_umap["kickType"].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "D", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "D", "component_2"
    ],
    s=5,
    alpha=0.5,
    color="lightgrey",
    label="Deep",
)

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "O", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "O", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Obvious onside",
)


# Hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.legend();


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "D", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "D", "component_2"
    ],
    s=5,
    alpha=0.5,
    color="lightgrey",
    label="Deep",
)

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "F", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "F", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Flat",
)

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "O", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "O", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Obvious onside",
)

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "P", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "P", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Pooch",
)

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "Q", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "Q", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Squib",
)

ax.scatter(
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "K", "component_1"
    ],
    defense_kickoff_umap.loc[
        defense_kickoff_umap["kickType"] == "K", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Free Kick",
)

# Hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.legend();


## Punt -- Blocked punts defense

In [ ]:
defense_punt_umap["specialTeamsResult"].value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

result_type = "Blocked Punt"

ax.scatter(
    defense_punt_umap.loc[
        defense_punt_umap["specialTeamsResult"] != result_type, "component_1"
    ],
    defense_punt_umap.loc[
        defense_punt_umap["specialTeamsResult"] != result_type, "component_2"
    ],
    s=5,
    alpha=0.5,
    color="lightgrey",
    label="All punt results (not blocked)",
)
ax.scatter(
    defense_punt_umap.loc[
        defense_punt_umap["specialTeamsResult"] == result_type, "component_1"
    ],
    defense_punt_umap.loc[
        defense_punt_umap["specialTeamsResult"] == result_type, "component_2"
    ],
    s=10,
    label="Blocked Punt",
)

# Hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


## Punt -- Blocked punts offense

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

result_type = "Blocked Punt"

ax.scatter(
    offense_punt_umap.loc[
        offense_punt_umap["specialTeamsResult"] != result_type, "component_1"
    ],
    offense_punt_umap.loc[
        offense_punt_umap["specialTeamsResult"] != result_type, "component_2"
    ],
    s=5,
    alpha=0.5,
    color="lightgrey",
    label="All punt results (not blocked)",
)
ax.scatter(
    offense_punt_umap.loc[
        offense_punt_umap["specialTeamsResult"] == result_type, "component_1"
    ],
    offense_punt_umap.loc[
        offense_punt_umap["specialTeamsResult"] == result_type, "component_2"
    ],
    s=10,
    label="Blocked Punt",
)

# Hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


## kickType for punts

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# normal style
ax.scatter(
    defense_punt_umap.loc[
        defense_punt_umap["kickType"] == "N", "component_1"
    ],
    defense_punt_umap.loc[
        defense_punt_umap["kickType"] == "N", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Normal",
    color="lightgrey",
)
# aussie style
ax.scatter(
    defense_punt_umap.loc[
        defense_punt_umap["kickType"] == "A", "component_1"
    ],
    defense_punt_umap.loc[
        defense_punt_umap["kickType"] == "A", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Aussie Style"
)
ax.scatter(
    defense_punt_umap.loc[
        defense_punt_umap["kickType"] == "R", "component_1"
    ],
    defense_punt_umap.loc[
        defense_punt_umap["kickType"] == "R", "component_2"
    ],
    s=5,
    alpha=0.5,
    label="Rugby"
)

# Hide the right and top spines
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.legend();
